# Прогноз дохода клиентов

 - Задача прогнозирования дохода имеет особенное значение в Банке.
Эта информация помогает точнее и более релевантно подбирать продукты и условия их приобретения, что в свою очередь вносит существенный вклад в прибыль Банка.
 - Помимо ценности для Банка, оценка дохода является регуляторным требованием ЦБ в части расчета предельно допустимой кредитной нагрузки для клиента (далее ПДН)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# здесь меняю рабочую папку, чтобы путь не прописывать к ней все время
import os
for dirpath, dirnames, _ in os.walk('/content/drive/MyDrive'):
    if 'hackathon' in dirnames:
        os.chdir(os.path.join(dirpath, 'hackathon'))
        break

# Импорт библиотек

In [196]:
import pandas as pd
import numpy as np

In [4]:
def weighted_mean_absolute_error(y_true, y_pred, weights):
    return (weights * np.abs(y_true - y_pred)).mean()

# Пример расчета метрики качества

In [174]:
train_df = pd.read_csv('hackathon_income_train.csv', decimal=',', sep=';')
test_df = pd.read_csv('hackathon_income_test.csv', decimal=',', sep=';')
train_df.shape, test_df.shape

/tmp/ipython-input-3956870551.py:1: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('hackathon_income_train.csv', decimal=',', sep=';')
/tmp/ipython-input-3956870551.py:2: DtypeWarning: Columns (65) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('hackathon_income_test.csv', decimal=',', sep=';')


((76786, 224), (73214, 222))

In [175]:
train_df.head(5)

,id,dt,target,turn_cur_cr_avg_act_v2,salary_6to12m_avg,hdb_bki_total_max_limit,dp_ils_paymentssum_avg_12m,hdb_bki_total_cc_max_limit,incomeValue,gender,...,dp_ils_uniq_companies_1y,avg_6m_travel,avg_6m_government_services,hdb_bki_active_cc_max_overdue,total_rur_amt_cm_avg_period_days_ago_v2,label_Above_1M_share_r1,transaction_category_supermarket_sum_cnt_d15,max_balance_rur_amt_1m_af,w,first_salary_income
0,2,2024-04-30,109324.476325,1465144.960,NaN,52800.0,365346.244634,23213.0,97366.0,Женский,...,1.0,0.0,57.0,0.0,297.0,0.027027,9.0,NaN,0.301217,NaN
1,4,2024-02-29,25558.028662,303593.660,NaN,260200.0,NaN,10000.0,32580.0,Женский,...,NaN,0.0,707.0,67.0,30245.0,NaN,2.0,NaN,0.695800,NaN
2,5,2024-02-29,40666.753098,490754.010,NaN,2000000.0,NaN,90000.0,96866.0,Женский,...,NaN,422.0,0.0,0.0,210322.0,0.000000,20.0,NaN,0.515970,NaN
3,6,2024-04-30,43856.672058,219875.120,NaN,75000.0,NaN,75000.0,43860.0,Мужской,...,NaN,0.0,0.0,0.0,7187.0,0.000000,7.0,NaN,0.478003,NaN
4,7,2024-04-30,130420.851992,1750241.845,NaN,1000000.0,NaN,240000.0,83815.0,Женский,...,NaN,0.0,84.0,0.0,690038.0,0.000000,9.0,NaN,0.552314,NaN


In [176]:
X = train_df.drop(columns=['target'])
y = train_df['target']

In [177]:
print(f'Общее кол-во столбцов {X.shape[1]}')
print(X.dtypes.value_counts())

Общее кол-во столбцов 223
float64    181
object      41
int64        1
Name: count, dtype: int64


In [178]:
# эти признаки нам не нужны
X.drop(columns=['id', 'dt'], inplace=True)

In [179]:
X = X.loc[:, X.isnull().mean() < 0.5] # оставим только столбцы, где пропусков меньше 50%

In [180]:
object_cols = X.select_dtypes(include='object').columns
X[object_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76786 entries, 0 to 76785
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   hdb_bki_total_max_limit         67413 non-null  object
 1   hdb_bki_total_cc_max_limit      63041 non-null  object
 2   gender                          76786 non-null  object
 3   adminarea                       57223 non-null  object
 4   hdb_bki_total_pil_max_limit     62001 non-null  object
 5   hdb_bki_active_cc_max_limit     56866 non-null  object
 6   city_smart_name                 67323 non-null  object
 7   hdb_bki_total_products          70561 non-null  object
 8   hdb_bki_total_max_overdue_sum   67257 non-null  object
 9   addrref                         68237 non-null  object
 10  bki_total_auto_cnt              64965 non-null  object
 11  hdb_bki_total_cnt               70483 non-null  object
 12  bki_total_oth_cnt               64965 non-null

In [181]:
categorical_cols = []
for col in object_cols:
    try:
      pd.to_numeric(X[col].dropna())
    except ValueError:
      categorical_cols.append(col)

In [182]:
categorical_cols

['gender', 'adminarea', 'city_smart_name', 'addrref']

In [183]:
# категориальные данные преобразовываем в one-hot, nan игнорим
X = pd.get_dummies(X, columns=categorical_cols, dummy_na=True)

In [184]:
# все оставшиеся столбцы преобразовываем в числовый и заполняем пропуски медианой
for col in X.select_dtypes(include=('object', 'float64')).columns:
    if X[col].dtype == 'object':
        X[col] = pd.to_numeric(X[col])
    X[col] = X[col].fillna(X[col].median())

In [185]:
X.isnull().sum().sum() # общее кол-во nan во всем датафрейме

np.int64(0)

In [186]:
print(f'Общее кол-во столбцов {X.shape[1]}')
print(X.dtypes.value_counts())

Общее кол-во столбцов 392
bool       258
float64    134
Name: count, dtype: int64


In [187]:
# избавляемся от столбцов с маленькой дисперсий
from sklearn.feature_selection import VarianceThreshold

numeric_cols = X.select_dtypes(include='float64').columns

selector = VarianceThreshold(threshold=0.01)
X_numeric_selected = selector.fit_transform(X[numeric_cols])

X[numeric_cols].columns[selector.get_support()]

X = X.drop(columns=numeric_cols[~selector.get_support()])

In [188]:
X.shape

(76786, 389)

In [189]:
# найдем столбцы, где присутсвуют выбросы
numeric_cols = X.select_dtypes(include='float64').columns

outliers = {}
for col in numeric_cols:
    q1 = X[col].quantile(0.25)
    q3 = X[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    count = ((X[col] < lower_bound) | (X[col] > upper_bound)).sum()

    outliers[col] = count

for col, count in outliers.items():
    if count > 0:
        print(f"{col}: {count}")

turn_cur_cr_avg_act_v2: 8111
hdb_bki_total_max_limit: 7163
hdb_bki_total_cc_max_limit: 6885
incomeValue: 5542
avg_cur_cr_turn: 8692
turn_cur_cr_avg_v2: 8836
turn_cur_cr_max_v2: 11517
hdb_bki_total_pil_max_limit: 7507
age: 361
turn_cur_cr_sum_v2: 8836
turn_cur_db_sum_v2: 8842
turn_cur_db_avg_act_v2: 8188
curr_rur_amt_cm_avg: 9780
turn_cur_db_avg_v2: 8842
avg_cur_db_turn: 8693
hdb_bki_active_cc_max_limit: 7689
incomeValueCategory: 878
avg_by_category__amount__sum__cashflowcategory_name__vydacha_nalichnyh_v_bankomate: 24948
avg_credit_turn_rur: 9551
by_category__amount__sum__eoperation_type_name__perevod_po_nomeru_telefona: 37683
turn_cur_cr_7avg_avg_v2: 8271
curbal_usd_amt_cm_avg: 12754
avg_by_category__amount__sum__cashflowcategory_name__supermarkety: 11702
avg_by_category__amount__sum__cashflowcategory_name__gipermarkety: 13270
turn_cur_db_max_v2: 11310
avg_by_category__amount__sum__cashflowcategory_name__kafe: 12519
turn_other_db_max_v2: 13006
turn_cur_cr_min_v2: 8060
turn_cur_db_min_

In [190]:
# Обрезаем, чтобы не было выбросов
X[numeric_cols] = X[numeric_cols].clip(
    lower=X[numeric_cols].quantile(0.01),
    upper=X[numeric_cols].quantile(0.99),
    axis=1
)

In [201]:
# избавляемся от мультколлинеарности
import itertools

correlation_matrix = X[numeric_cols].corr()

corr_pairs = correlation_matrix.unstack()
corr_pairs = corr_pairs[corr_pairs < 1]
high_corr = corr_pairs[abs(corr_pairs) > 0.8].sort_values(ascending=False)
print(high_corr)

mob_cnt_days         mob_cover_days         1.000000
mob_cover_days       mob_cnt_days           1.000000
turn_cur_db_avg_v2   turn_cur_db_sum_v2     1.000000
turn_cur_db_sum_v2   turn_cur_db_avg_v2     1.000000
turn_cur_cr_avg_v2   turn_cur_db_avg_v2     0.998780
                                              ...   
turn_cur_db_sum_v2   avg_credit_turn_rur    0.802312
avg_credit_turn_rur  turn_cur_db_avg_v2     0.802312
turn_cur_db_avg_v2   avg_credit_turn_rur    0.802312
dda_rur_amt_curr_v2  dda_rur_amt_3m_avg     0.800671
dda_rur_amt_3m_avg   dda_rur_amt_curr_v2    0.800671
Length: 162, dtype: float64


In [ ]:
# выбросов много, поэтому в качестве стандартизации используем RobustScaler

# Submission

In [ ]:
test_df['target'] = np.random.normal(mean, std, test_df.shape[0])
test_df[['id','target']].set_index('id').to_csv('submission.csv', decimal='.', sep=',')